# Crawler基礎－select、multi-page

## 目錄
- [select](#select)
- [multi-page](#multiple-page)

### select

使用 css selector 來選取元素
- [W3school - CSS_selector](https://www.w3schools.com/cssref/css_selectors.asp)

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# 先把後面會用到的全域變數放在這裡
url = "http://pala.tw/class-id-example/"
headers = {'user-agent':'Safari'}

In [3]:
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [4]:
# 使用 css selector 搜尋
found_list = soup.select('#Pala')
found_list

[<p class="無糖 微冰 鐵觀音" id="Pala">Pala的無糖微冰鐵觀音</p>]

In [5]:
# select 不論結果是一個或是多個
# 都會用 list 回傳
found_list = soup.select('.去冰')
found_list

[<p class="半糖 去冰 珍珠 奶茶" id="老闆">老闆的半糖去冰珍珠奶茶</p>,
 <p class="無糖 去冰 綠茶" id="資深前輩">資深前輩的無糖去冰綠茶</p>,
 <p class="半糖 去冰 文山青茶" id="心儀同事">心儀同事的半糖去冰文山青茶</p>]

In [6]:
# 把需要的字串擷取出來
str_list = [each.string for each in found_list]
str_list

['老闆的半糖去冰珍珠奶茶', '資深前輩的無糖去冰綠茶', '心儀同事的半糖去冰文山青茶']

In [7]:
# 取出特別的屬性
get_list = [each.get('id') for each in found_list]
get_list

['老闆', '資深前輩', '心儀同事']

解析HTML的方法差不多就到這裡
常用的三種解析方法：
1. 套件內建的function
2. css selector
3. Xpath
有需要可以再自己上網學習

## multiple-page

如果多練習幾次爬蟲，多使用requests和bs套件  
你會發現他有一些固定的規律  
如果我們想要一次性多抓取一些頁面  
就很適合設計成function  
讓程式碼變的簡潔

In [10]:
# 把 url 傳進來
# 用 requests 抓取後 回傳原始碼
def get_page(url):
    head = {'user-agent':'Safari'}
    res = requests.get(url,headers=head)
    return(res.text)

In [12]:
# 先用 temp 抓取到所有新聞的網頁原始碼
temp = get_page('https://news.pts.org.tw/list/0')

In [15]:
# 利用 bs 選出 title 做成 list
soup = BeautifulSoup(temp,'html.parser')
title_list = soup.find_all(class_='list-news-title')
title_list = [each.find('a') for each in title_list]

In [20]:
# 選到了 我們用前五個就好
title_list = title_list[0:5]
title_list

[<a href="https://news.pts.org.tw/article/378162">李明哲遭中國判刑5年 當庭放棄上訴</a>,
 <a href="https://news.pts.org.tw/article/378163">新北之外五都補助私幼學費 民團表抗議</a>,
 <a href="https://news.pts.org.tw/article/378164">歐洲議會表決通過 展延"嘉磷賽"執照5年</a>,
 <a href="https://news.pts.org.tw/article/378165">雞蛋"芬普尼"農藥容許量 放寬至10ppb</a>,
 <a href="https://news.pts.org.tw/article/378166">勞團:勞基修法前 多工會被打壓.成員遭解僱</a>]

In [26]:
# 先創個 out 資料夾
from os import makedirs
makedirs('out',exist_ok=True)

In [29]:
# 利用get_page函式 存每個頁面
for title in title_list:
    res = get_page(title.get('href'))
    
    file_name = 'out/' + title.string + '.html'
    html = open(file_name,'w')
    html.write(res)
    html.close()

In [52]:
# 再來寫個負責parser的func吧!
def get_content(html):
    soup = BeautifulSoup(html,'html.parser')
    
    content = soup.find(class_='article-content')
    return(content)

In [64]:
for title in title_list:
    file_name = 'out/' + title.string
    
    html = open(file_name + '.html','r')
    content = get_content(html.read())
    html.close()
    
    txt = open(file_name + '.txt','w')
    txt.write(content.get_text())
    txt.close()

In [68]:
# 更快速的方法 不存成檔案 直接放到parser裡面
for title in title_list:
    html = get_page(title.get('href'))
    content = get_content(html)
    
    print(content.get_text()[0:50])
    print()

台灣NGO工作者李明哲今天被中國政府以顛覆國家政權罪判處5年徒刑，李明哲當庭表示接受判決也不會上訴，

除了新北市之外其他五都，近來紛紛宣布從明年起、補助私立幼兒園學費，而且補助對象還向下延伸到2歲，今天

台灣用很多的除草劑年年春，裡面的 嘉磷賽"，被世界衛生組織認定為可能致癌物。由於"嘉磷賽"在歐洲的使

雞蛋是否要制定農藥芬普尼殘留容許量？衛福部與農委會開跨部會專家會議，經過討論達成共識。考量蛋雞飼料進

勞動法修法要鬆綁一例一休，將彈性工時交由勞資協商，不過，勞工團體則出面控訴，在修法前，已經有多個工會



In [69]:
# 世界快
content_list = [get_content(get_page(title.get('href'))).get_text()
               for title in title_list]
content_list

['台灣NGO工作者李明哲今天被中國政府以顛覆國家政權罪判處5年徒刑，李明哲當庭表示接受判決也不會上訴，李明哲的妻子李凈瑜也到場親自聽判，她對判決會面對。陸委會和總統府也發表聲明，無法接受判決內容，呼籲北京當局儘速放人。\n\r\n==中國法院宣判==\r\n被告人李明哲犯顛覆國家政權罪\r\n判處有期徒刑5年\r\n剝奪政治權利2年\n\r\n面無表情聽著判決，台灣NGO工作者李明哲，28號遭到中國政府以顛覆國家政權判處五年徒刑，並剝奪政治權力2年，李明哲當庭認罪表示不會上訴。\n\r\n==NGO工作者 李明哲==\r\n（被告人李明哲)\r\n（是否服從判決 是否上訴)\r\n服從判決 不上訴\n\r\n李明哲已經在中國被羈押超過250天，28號一早中國岳陽中級人民法院9點15分開庭，李明哲妻子李凈瑜也到場親自聽判，神情嚴肅，隨後李明哲和彭宇華被帶進法庭，兩人都被認定以網路和聚會惡意攻擊中國政府，涉嫌顛覆國家政權，彭宇華被判7年，李明哲5年,兩人都服從法院判決，不到半小時宣判結束就被帶離法庭。\n\r\n總統府表示，傳播民主理念無罪,分享民主自由理念,卻遭到中國大陸法院以「顛覆國家」這種罪名審判，「我們無法接受」，並呼籲北京當局早日放人讓李明哲儘速返台。而我陸委會則對宣判內容，也表示無法接受。\n\r\n==陸委會發言人 邱垂正==\r\n我們對於這樣判決的結果\r\n我方各界沒有辦法接受\r\n與民主人權等普世價值\r\n也是完全背離的\r\n也勢必會對兩岸交流\r\n帶來嚴重的負面影響\n\r\n李明哲也是首位被以顛覆國家政權罪判刑確定的境外人士，各界都相當關注，李明哲妻子李凈瑜表示追求理想必須付出代價，以李明哲為榮，坦然面對。\n\r\n記者 綜合報導',
 '除了新北市之外其他五都，近來紛紛宣布從明年起、補助私立幼兒園學費，而且補助對象還向下延伸到2歲，今天有十多個教保民間團體召開記者會，抗議這種作法根本是變相買票，呼籲政府應多增設公幼或非營利幼兒園\n\r\n高喊口號、手持標語，全台十多個幼教民間團體召開記者會，抗議除了新北市外的其他五都，近來相繼宣布將加碼補助私立幼兒園學費，完全是本末倒置的做法，不僅是變相買票拉攏家長，而且錢只會進入業者荷包，對於幼教薪資待遇提升很有限，也無法改善幼教品質。\n\r\n==全國教保產業工會理事長 簡瑞連==\r\n